# HDL playground

This HDL playground is based on the `hackfin/myhdl_testing:jupyosys` Docker container. The underlying toolchain is the [yosys](https://github.com/YosysHQ/yosys/) synthesis toolchain.

It is meant to collect examples and test scenarios for various Hardware Definition Languages.

The top level default command language is Python, with frequent usage of MyHDL for test benches, co-simulation or HDL modules to be emitted as Verilog and/or VHDL implementation for functional comparison or verification.

Direct (experimental) Synthesis through MyHDL is supported by the [jupyosys](https://section5.ch/index.php/2020/03/20/myhdl-and-pyosys-direct-synthesis-using-python/) project, see separate Binder:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/hackfin/myhdl.git/jupyosys?filepath=src%2Fmyhdl%2Fmyhdl-yosys%2Fexample%2Fipynb%2Findex.ipynb)

Supported HDLs in order of maturity with respect to yosys:

* Verilog (simulation through iverilog)
* VHDL (simulation and synthesis using GHDL)
* MyHDL (through jupyosys)

Automated testing of notebook is integrated [below](#Automated-notebook-tests).

## Simulation issues

There is an issue with specific files that are required to simulate hardware primitives:

* They are vendor-proprietary
* Possibly under NDA or encrypted

If you want to co-simulate against a vendor entity, the current procedure is as follows:

1. Obtain the `*.v` files needed for simulation from your installed toolchain
+  Copy those files into the directory specified by the `ECP5_LIB` variable in [config.py](/edit/work/tests/config.py),
see also Upload link below.

An 'enhanced' `cells_sim.v` file is added for example that includes some vendor simulation files (click to edit: [cells_sim.v](/edit/work/tests/lib/techlibs/ecp5/cells_sim.v)). You need to uncomment the lines on the bottom after installation of the necessary `*.v` files (use the `Upload` button on this [directory](/tree/work/tests/lib/techlibs/ecp5)).

**Note**: The vendor simulation files should not be included in yosys, as they contain non-synthesizable Verilog
statements. For yosys, these entities appear as blackbox cells, later on, the icarus verilog simulator
includes them for proper co-simulation.

## VHDL support

The GHDL yosys plugin is automatically compiled during the build process of this container.

Proceed to [GHDL overview](ghdl.ipynb)

# Automated notebook tests

Automated notebook tests are run with the code below to verify that:

* There is no exception occuring during execution
* The output matches the previous notebook output

It may be that a notebook shows a `FAILED` state and passes upon regeneration and the next autotest run. For strict comparison, remove `--nb-force-regen` below.

Note that some tests may also fail when the above vendor files are not installed.

In [1]:
! pip3 install pytest-notebook >/dev/null

Install the magic extension:

In [2]:
%load_ext pytest_notebook.ipy_magic

We have to manually configure the pytest executable:

In [3]:
import pytest_notebook
pytest_notebook.ipy_magic.EXEC_NAME = "py.test-3"

### Running the test suite

In [4]:
%%pytest -v --color=yes --disable-warnings --nb-exec-timeout 50 --nb-force-regen

---
[pytest]
nb_test_files = True
nb_diff_ignore = 
    /cells/*/outputs/*/data/image/svg+xml
nb_diff_replace =
    /cells/*/outputs/*/data/text .*graphviz\.files\.Source.* "GRAPH"
    /cells/*/outputs/*/text \[SigSpec.*\] "SIGSPEC"
    /cells/*/outputs/*/text .*\.vcd "VCDFILE"
    /cells/*/outputs/*/data/text .*graphviz\.dot\.Digraph.* "GRAPH"
    /cells/*/outputs/*/text .*-rw\-r\-\-r\-\-.* "FILE"
    /cells/*/outputs/*/text .*End.of.script.* "YOSYS_TAG"

---

import sys
import importlib_resources

sys.path.append("/home/pyosys/work")
sys.path.append("/home/pyosys/work/examples")

import pytest
import tests
import tempfile

        
VERIFIED  = [ "ghdl_module_tests.ipynb", "ghdlsim_blink.ipynb", "memory.ipynb" , "memory_wrapped_init.ipynb" ]

KNOWN_TO_FAIL = [ ]

@pytest.mark.parametrize("uut", VERIFIED)
def test_generic(nb_regression, uut):
    with importlib_resources.path(tests, uut) as path:
        nb_regression.check(str(path))                

@pytest.mark.xfail 
@pytest.mark.parametrize("uut", KNOWN_TO_FAIL)
def test_fixme(nb_regression, uut):
    with importlib_resources.path(tests, uut) as path:
        nb_regression.check(str(path))                              

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-3.10.1, py-1.7.0, pluggy-0.8.0 -- /usr/bin/python3
cachedir: .pytest_cache
NB force regen: True
rootdir: /tmp/tmpqdjn2ocw, inifile: pytest.ini
plugins: notebook-0.6.0
collecting ... collected 5 items

test_ipycell.py::test_generic[ghdl_module_tests.ipynb] PASSED            [ 20%]
test_ipycell.py::test_generic[ghdlsim_blink.ipynb] PASSED                [ 40%]
test_ipycell.py::test_generic[memory.ipynb] PASSED                       [ 60%]
test_ipycell.py::test_generic[memory_wrapped_init.ipynb] PASSED          [ 80%]
test_ipycell.py::test_fixme[uut0] SKIPPED                                [100%]

===================== 4 passed, 1 skipped in 43.36 seconds =====================
